 # ORB Detector Parameter Tuning 
## Setup the required functions and input images

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Feb  1 20:06:06 2019

@author: vik748
"""
import numpy as np
import cv2
import sys
from matplotlib import pyplot as plt
from ssc import *
%matplotlib inline
from scipy import spatial
import time
np.set_printoptions(precision=4,suppress=True)

#from mahotas import features

def draw_keypoints(vis_orig, keypoints, color = (255, 255, 0), thick = 1):
    vis = vis_orig.copy()
    for kp in keypoints:
        x, y = kp.pt
        cv2.circle(vis, (int(x), int(y)), int(vis.shape[1]/200), color, thickness=thick)
    return vis

def draw_markers(vis_orig, keypoints, color = (0, 0, 255)):
    vis = vis_orig.copy()
    for kp in keypoints:
        x, y = kp.pt
        cv2.drawMarker(vis, (int(x), int(y)), color,  markerSize=30, markerType = cv2.MARKER_CROSS, thickness=3)
    return vis

def draw_marker_pts(vis_orig, pts, color = (255, 255, 0),markerSize=30):
    vis = cv2.cvtColor(vis_orig,cv2.COLOR_GRAY2RGB)
    for row in pts[:,0,:]:
        cv2.drawMarker(vis, (int(row[0]), int(row[1])), color,  markerSize=markerSize, markerType = cv2.MARKER_CROSS, thickness=1)
    return vis

def draw_arrows(vis_orig, points1, points2, color = (0, 255, 0), thick = 1):
    vis = cv2.cvtColor(vis_orig,cv2.COLOR_GRAY2RGB)
    #rad = int(vis.shape[1]/200)
    for p1,p2 in zip(points1,points2):
        cv2.arrowedLine(vis, (int(p1[0]),int(p1[1])), (int(p2[0]),int(p2[1])), color=(0,255,0), thickness=thick)
    return vis

def draw_feature_tracks(img_left,kp1,img_right,kp2, matches, mask, display_invalid=False, color=(0, 255, 0)):
    '''
    This function extracts takes a 2 images, set of keypoints and a mask of valid
    (mask as a ndarray) keypoints and plots the valid ones in green and invalid in red.
    The mask should be the same length as matches
    '''
    if mask is None: bool_mask = np.ones((len(matches)), dtype=bool)
    else: bool_mask = mask.astype(bool)
    valid_right_matches = np.array([kp2[mat.trainIdx].pt for is_match, mat in zip(bool_mask, matches) if is_match])
    valid_left_matches = np.array([kp1[mat.queryIdx].pt for is_match, mat in zip(bool_mask, matches) if is_match])
    #img_right_out = draw_points(img_right, valid_right_matches)
    img_right_out = draw_arrows(img_right, valid_left_matches, valid_right_matches)
    return img_right_out

def displayMatches(img_left,kp1,img_right,kp2, matches, mask, display_invalid, in_image=None, color=(0, 255, 0)):
    '''
    This function extracts takes a 2 images, set of keypoints and a mask of valid
    (mask as a ndarray) keypoints and plots the valid ones in green and invalid in red.
    The mask should be the same length as matches
    '''
    bool_mask = mask.astype(bool)
    if in_image is None: mode_flag=0
    else: mode_flag =1
    img_valid = cv2.drawMatches(img_left,kp1,img_right,kp2,matches, in_image, 
                                matchColor=color, 
                                matchesMask=bool_mask.ravel().tolist(), flags=mode_flag)
    
    if display_invalid:
        img_valid = cv2.drawMatches(img_left,kp1,img_right,kp2,matches, img_valid, 
                                  matchColor=(255, 0, 0), 
                                  matchesMask=np.invert(bool_mask).ravel().tolist(), 
                                  flags=1)
    return img_valid


K = np.array([[699.33112889, 0.0, 403.23876197],
              [0.0, 693.66457792, 293.40739086],
              [0.0, 0.0, 1.0]])

D =  np.array([-2.78089511e-01,  1.30037134e-01, -1.17555797e-04, -1.55337290e-04, -4.34486330e-02])

if sys.platform == 'darwin':
    path = '/Users/vik748/Google Drive/'
else:
    path = '/home/vik748/'
img1 = cv2.imread(path+'data/time_lapse_5_cervino_800x600/G0057821.png',1) # iscolor = CV_LOAD_IMAGE_GRAYSCALE
img2 = cv2.imread(path+'data/time_lapse_5_cervino_800x600/G0057826.png',1) # iscolor = CV_LOAD_IMAGE_GRAYSCALE


gr1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
gr2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
'''
clahe = cv2.createCLAHE(clipLimit=40.0, tileGridSize=(4,3))
gr1 = clahe.apply(gr1)
gr2 = clahe.apply(gr2)
'''

# create a mask image filled with 1s, the size of original image
m1 = cv2.imread(path+'data/time_lapse_5_cervino_800x600_masks_out/G0056825_mask.png',cv2.IMREAD_GRAYSCALE)
m2 = cv2.imread(path+'data/time_lapse_5_cervino_800x600_masks_out/G0057826_mask.png',cv2.IMREAD_GRAYSCALE)

def radial_non_max(kp_list, dist):
    kp = np.array(kp_list)
    kp_mask = np.ones(len(kp), dtype=bool)
    pts = [k.pt for k in kp]
    tree = spatial.cKDTree(pts)
    #print ("len of kp1:",len(kp))
    for i, k in enumerate(kp):
        if kp_mask[i]:
            pt = tree.data[i]
            idx = tree.query_ball_point(tree.data[i], dist, p=2., eps=0, n_jobs=1)
            resp = [kp[ii].response for ii in idx]
            _, maxi = max([(v,i) for i,v in enumerate(resp)])
            del idx[maxi]
            for kp_i in idx:
                kp_mask[kp_i] = False 
    return kp[kp_mask].tolist()

def bounding_box(points, min_x=-np.inf, max_x=np.inf, min_y=-np.inf,
                        max_y=np.inf):
    """ Compute a bounding_box filter on the given points

    Parameters
    ----------                        
    points: (n,2) array
        The array containing all the points's coordinates. Expected format:
            array([
                [x1,y1],
                ...,
                [xn,yn]])

    min_i, max_i: float
        The bounding box limits for each coordinate. If some limits are missing,
        the default values are -infinite for the min_i and infinite for the max_i.

    Returns
    -------
    bb_filter : boolean array
        The boolean mask indicating wherever a point should be keept or not.
        The size of the boolean mask will be the same as the number of given points.

    """

    bound_x = np.logical_and(points[:, 0] > min_x, points[:, 0] < max_x)
    bound_y = np.logical_and(points[:, 1] > min_y, points[:, 1] < max_y)

    bb_filter = np.logical_and(bound_x, bound_y)

    return bb_filter


def tiled_features(kp, img_shape, tiley, tilex):
    feat_per_cell = int(len(kp)/(tilex*tiley))
    HEIGHT, WIDTH = img_shape
    assert WIDTH%tiley == 0, "Width is not a multiple of tilex"
    assert HEIGHT%tilex == 0, "Height is not a multiple of tiley"
    w_width = int(WIDTH/tiley)
    w_height = int(HEIGHT/tilex)
        
    xx = np.linspace(0,HEIGHT-w_height,tilex,dtype='int')
    yy = np.linspace(0,WIDTH-w_width,tiley,dtype='int')
        
    kps = np.array([])
    pts = np.array([keypoint.pt for keypoint in kp])
    kp = np.array(kp)
    
    for ix in xx:
        for iy in yy:
            inbox_mask = bounding_box(pts, iy,iy+w_height,ix,ix+w_height)
            inbox = kp[inbox_mask]
            inbox_sorted = sorted(inbox, key = lambda x:x.response, reverse = True)
            inbox_sorted_out = inbox_sorted[:feat_per_cell]
            kps = np.append(kps,inbox_sorted_out)
    return kps.tolist()


## Feature Detection

In [ ]:
TILE_KP = True
RADIAL_NON_MAX = True
RADIAL_NON_MAX_RADIUS = 5
TILEY = 4
TILEX = 3
'''
detector = cv2.ORB_create(nfeatures=10000, scaleFactor=2.0, nlevels=4, edgeThreshold=31, 
                           firstLevel=0, WTA_K=2, scoreType=cv2.ORB_HARRIS_SCORE,  
                           patchSize=31, fastThreshold=15)
'''
detector = cv2.xfeatures2d.SIFT_create(nfeatures = 10000, nOctaveLayers = 3, contrastThreshold = 0.01, 
                                       edgeThreshold = 20, sigma = 1.6)
# find the keypoints and descriptors with ORB
kp1 = detector.detect(gr1,m1)
print("Detected kps: ",len(kp1))
kp2 = detector.detect(gr2,m2)

if TILE_KP:
    kp1 = tiled_features(kp1, gr1.shape, TILEY, TILEX)
    kp2 = tiled_features(kp2, gr2.shape, TILEY, TILEX)
    print ("Points after tiling supression: ",len(kp1))

if RADIAL_NON_MAX:
    kp1 = radial_non_max(kp1,RADIAL_NON_MAX_RADIUS)
    kp2 = radial_non_max(kp2,RADIAL_NON_MAX_RADIUS)
    print ("Points after radial supression: ",len(kp1))

img_out1 = draw_keypoints(cv2.cvtColor(gr1, cv2.COLOR_GRAY2BGR),kp1)
img_out2 = draw_keypoints(cv2.cvtColor(gr2, cv2.COLOR_GRAY2BGR),kp2)


fig, [ax1, ax2] = plt.subplots(1,2,dpi=200)
fig.suptitle('800x600 Yellow circle defult setting patchSize=31')
ax1.axis("off")
ax1.imshow(img_out1)
ax2.axis("off")
ax2.imshow(img_out2)
fig.subplots_adjust(0,0,1,1,0.0,0.0)
plt.show()

## Feature matching

In [ ]:
#kp1, des1 = detector.compute(gr1,kp1)
#kp2, des2 = detector.compute(gr2,kp2)

kp1z=[]
des1z = np.zeros((len(kp1),25),dtype='float32')
for i,k in enumerate(kp1):
    mts = features.zernike_moments(gr1, 22, 8,cm=k.pt)
    if mts.any():
        des1z[i,:]=mts
        kp1z += [k]
des1z = des1z[:len(kp1z)]
   
kp2z=[]
des2z = np.zeros((len(kp2),25),dtype='float32')
for i,k in enumerate(kp2):
    mts = features.zernike_moments(gr2, 22, 8,cm=k.pt)
    if mts.any():
        des2z[i,:]=mts
        kp2z += [k]    
des2z = des2z[:len(kp2z)]

    
'''
FLANN_INDEX_LSH = 6
FLANN_INDEX_KDTREE = 1
matcher = cv2.FlannBasedMatcher(dict(algorithm = FLANN_INDEX_LSH, table_number = 6, key_size = 20,
                                   multi_probe_level = 2), dict(checks=100))
'''
matcher = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
#matches12_knn = matcher.knnMatch(des1,des2,k=2)
matches12 = matcher.match(des1z,des2z)
#matches12 = [item for sublist in matches12_knn for item in sublist]
print("Found matches: ",len(matches12))

kp1_match_12 = np.array([kp1z[mat.queryIdx].pt for mat in matches12])
kp2_match_12 = np.array([kp2z[mat.trainIdx].pt for mat in matches12])

#matches12 = sorted(matches12, key = lambda x:x.distance)
#matches12 = matches12[:(int)(len(matches12)*.75)]

kp1_match_12_ud = cv2.undistortPoints(np.expand_dims(kp1_match_12,axis=1),K,D)
kp2_match_12_ud = cv2.undistortPoints(np.expand_dims(kp2_match_12,axis=1),K,D)

E_12, mask_e_12 = cv2.findEssentialMat(kp1_match_12_ud, kp2_match_12_ud, focal=1.0, pp=(0., 0.), 
                                       method=cv2.RANSAC, prob=0.9999, threshold=0.001)

print("After essential: ", np.sum(mask_e_12))

#img_valid = draw_feature_tracks(gr1,kp1z,gr2,kp2z, matches12, mask_e_12, display_invalid=True, color=(0, 255, 0))
img_valid = displayMatches(gr1,kp1,gr2,kp2, matches12, mask_e_12, display_invalid=True, color=(0, 255, 0))

fig, ax= plt.subplots(dpi=200)
plt.title('800x600 Yellow circle defult setting patchSize=31')
plt.axis("off")
plt.imshow(img_valid)
plt.show()


In [ ]:
    matches_knn = matcher.knnMatch(des1,des2, k=5)
    matches = []
    kp1_match = []
    kp2_match = []
    
    for i,match in enumerate(matches_knn):
        if len(match)>1:
            if match[0].distance < 0.80*match[1].distance:
                matches.append(match[0])
                kp1_match.append(kp1[match[0].queryIdx].pt)
                kp2_match.append(kp2[match[0].trainIdx].pt)
        elif len(match)==1:
            matches.append(match[0])
            kp1_match.append(kp1[match[0].queryIdx].pt)
            kp2_match.append(kp2[match[0].trainIdx].pt)


In [ ]:
class MultiHarrisZernike (cv2.Feature2D):
    def __init__(self, first, last, staffnum):
        Person.__init__(self,first, last)
        self.staffnumber = staffnum

In [ ]:
a = MultiHarrisZernike()

In [ ]:
a.detect(gr1)

In [ ]:
from scipy import signal
I = np.float64(gr1)

dx = 
signal.convolve2d(dx,dx,'same')

In [ ]:
#from scipy.signal import convolve2d
from scipy.ndimage.filters import maximum_filter
from scipy.ndimage import convolve

def fspecial_gauss(size, sigma):
    """
    Function to mimic the 'fspecial' gaussian MATLAB function
    """
    x, y = np.mgrid[-size//2 + 1:size//2 + 1, -size//2 + 1:size//2 + 1]
    g = np.exp(-((x**2 + y**2)/(2.0*sigma**2)))
    return g/g.sum()

def harris(img, N_feat = 500, thresh = 0.0, tiling = (1,1), mask=None,
           gsize = 3, gsigma = 0.5, local_maxima_nhood = 3):
    '''
    HARRIS Harris corner detector
    This function implements a version of the Harris corner detector which
    has the ability to calculate the eigenvalues of the gradient matrix
    directly.  This is opposed to calculating the corner response function as
    originally proposed by Harris in:
    
    C. Harris and M. Stephens.  "A Combined Corner and Edge
    Detector", Proceedings of the 4th Alvey Vision Conference,
    Manchester, U.K. pgs 147-151, 1988
    
    INPUT:
    I is the graylevel image to extract interest points from
    PARAMETER     DEFAULT   DESCRIPTION
    ----------   --------  --------------------------
    N_feat            500    maximum number of interest points to return
    'thresh'          0    smallest acceptable value of response function
    'gsize          3    size of the smoothing Gaussian mask
    'gsigma         0.5   standard deviation of Gaussian filter
    local_maxima_nhood
    'tile'          [1 1]  break the image into regions [y x] to
                           distribute feature points more uniformly
    'mask'           M     array of 1's same size as image defining
                           where to compute feature points (useful
                           for radially compensated images)
    
    OUTPUT:
    kps=(y,x) are the row/column locations of interest points
    M is the corner response function value associated with that point
    
    Example:
    I = cv2.imread('cameraman.tif');
    [kps,M] = harris(I,500,tiling=(2,2))
    '''
    if len(img.shape) > 2:
        raise ValueError('Image supplied is not grayscale')
    
    border = int(np.ceil(gsize/2))
    mb = np.zeros_like(img)
    mb[border:-border,border:-border]=1
    if mask is None:
        mask = mb
    else:
        mask = np.logical_and(mask,mb)
        
    I = np.float64(img)
    (nr, nc) = I.shape
    (tile_x, tile_y) = tiling

    dx = np.array([[-1,0,1],[-1, 0, 1], [-1,0,1]])/3
    dy = dx.T
    
    st = time.time()
    Ix = convolve(I,dx,mode='nearest')
    Iy = convolve(I,dy,mode='nearest')
    #[Iy, Ix] = np.gradient(I,edge_order=1) - The way it was in Oscar's version

    IxIx = np.square(Ix)
    IyIy = np.square(Iy)
    IxIy = Ix*Iy

    gmask = fspecial_gauss(gsize, gsigma)
    IxIx = convolve(IxIx,gmask,mode='nearest')
    IyIy = convolve(IyIy,gmask,mode='nearest')
    IxIy = convolve(IxIy,gmask,mode='nearest')
    #print("elapsde =",time.time()-st )

    B = IxIx + IyIy
    SQRTRM = np.sqrt(np.square(B) - 4*(IxIx*IyIy-np.square(IxIy)))
    lambda1 = (B+SQRTRM)/2
    lambda2 = (B-SQRTRM)/2
    R = lambda1*lambda2 - 0.04*np.square(lambda1+lambda2)
    Maxima = (np.logical_and(maximum_filter(R,size=local_maxima_nhood)<=R, mask)) * R  # add mask here...
    (i,j) = np.nonzero(Maxima > thresh)
    m = Maxima[i,j]
    sidx = np.argsort(-m) # Sorted indexes
    m = m[sidx]
    i = i[sidx]
    j = j[sidx]

    w_width = int(nc/tile_x)
    w_height = int(nr/tile_y)

    if tile_x > 1 and tile_y > 1:
        if nc % tile_x >0 or nr % tile_y>0:
            raise ValueError('Image size not a multiple of specified tiling')
        Npts_per_region = int(np.round(N_feat/(tile_x*tile_y)))
        # process image regionally so that corners are uniformally
        # extracted across image regions
        ii = np.array([],dtype=int)
        jj = np.array([],dtype=int)     
        mm = np.array([],dtype=float)
        xx = np.round(np.linspace(0,nc,tile_x+1)) # region boundaries
        yy = np.round(np.linspace(0,nr,tile_y+1))
        for xll,xul in zip(xx[:-1],xx[1:]):
            # points falling within the region's x boundaries
            maskx = np.logical_and(j >= xll, j < xul)
            for yll,yul in zip(yy[:-1],yy[1:]):
                # points falling within the region's y boundaries
                masky = np.logical_and(i >= yll, i < yul)

                # their common intersection
                maskxy = np.logical_and(maskx, masky)
                id = np.argwhere(maskxy)
                sel_pts = id[0:Npts_per_region]
                #print(id)
                # return the strongest N points as defined by user
                #print (id[0:Npts_per_region])
                ii = np.append(ii, i[sel_pts])
                jj = np.append(jj, j[sel_pts])
                mm = np.append(mm, m[sel_pts])      
    else:
        ii = i[0:N_feat]
        jj = j[0:N_feat]
        mm = m[0:N_feat]
    return np.vstack((jj,ii)).T,mm


In [ ]:
Gi = fspecial_gauss(11, 2.75)
lpf = P.lpimages[0]
scale = 0.75
[eig, nL] = eigen_image_p(lpf,Gi,scale)
fig, [ax1,ax2] = plt.subplots(2,1,dpi=200)
fig.suptitle('eig and nL images')
#a.axis("off")
ax1.imshow(eig,cmap='gray'),ax1.axis("off")
ax2.imshow(nL,cmap='gray'),ax2.axis("off")
plt.show()

In [ ]:
st = time.time()
for i in range(10):
    kps, M = harris(gr1, N_feat = 500, thresh = 0.0, tiling = (8,6), mask=None,
               gsize = 3, gsigma = 0.5, local_maxima_nhood = 3)
print("elapsed: ",(time.time()-st)/10)
imgout = draw_marker_pts(gr1, np.expand_dims(kps,axis=1),markerSize=10)


fig, ax1 = plt.subplots(1,1,dpi=200)
plt.title('800x600 Yellow circle defult setting patchSize=31')
plt.axis("off")
plt.imshow(imgout)
plt.show()

In [ ]:
def eigen_image_p(lpf,Gi,scale):
    '''    
    %function [ef2,nL] = eigen_image_p(lpf,Gi)
    %set up in pyramid scheme with detection scaled smoothed images
    %ef2 is the interest point eigen image
    %lpf smoothed by the detection scale gaussian
    %Gi = fspecial('gaussian',ceil(7*sigi),sigi);
    '''
    [fy,fx] = np.gradient(lpf)

    [fxy,fxx] = np.gradient(fx)
    [fyy,fyx] = np.gradient(fy)
    nL = scale**(-2)*np.abs(fxx+fyy)

    Mfxx = convolve(np.square(fx),Gi,mode='constant')
    Mfxy = convolve(fx*fy,Gi,mode='constant')
    Mfyy = convolve(np.square(fy),Gi,mode='constant')

    Tr = Mfxx+Mfyy
    Det = Mfxx*Mfyy-np.square(Mfxy)
    sqrterm = np.sqrt(np.square(Tr)-4*Det)

    ef2 = scale**(-2)*0.5*(Tr - sqrterm)
    return ef2,nL

class ImagePyramid:
    def __init__ (self, levels, ratio, sigdec, sigint):
        self.levels = levels
        self.ratio = ratio
        self.images = []
        self.lpimages = []
        self.sigd = [sigdec]
        self.sigi = [sigint]
        self.lpfilter = fspecial_gauss(int(np.ceil(7*sigdec)),sigdec)
        # Make a note of this 7x7 gaussian kernel, lowering will speed up computation
        
    def generate_pyramid(self,img):
        self.images = [np.float64(img)]
        self.lpimages = [convolve(self.images[0],self.lpfilter,mode='constant')]
        for k in range(1,self.levels):
            self.images += [cv2.resize(self.lpimages[-1], (0,0), fx=self.ratio,
                                      fy=self.ratio, interpolation=cv2.INTER_LINEAR)]
            self.lpimages += [convolve(self.images[-1],self.lpfilter,mode='nearest')]
            self.sigd += [self.sigd[-1]/self.ratio] #equivalent sigdec at max res
            self.sigi += [self.sigi[-1]/self.ratio] 

            
P = ImagePyramid(6,0.75,1,2.75)
st = time.time()
for i in range(10):
    P.generate_pyramid(gr1)
print("time: ",(time.time()-st)/10)
fig, ax1 = plt.subplots(1,1,dpi=200)
plt.axis("off")
plt.imshow(P.lpimages[3],cmap='gray')
plt.show()

In [ ]:
P = ImagePyramid(6,0.75,1,2.75)
P.generate_pyramid(gr1)
zrad = 22
Gi = fspecial_gauss(11, 2.75)
local_maxima_nhood=3

scales = P.levels
ratio = P.ratio
#E(1).levels = scales;
border = zrad
[rows,cols] = P.lpimages[0].shape

eig = [None] * scales
nL = [None] * scales
border_mask = [None] * scales 
regmask=[None] * scales 
ivec= [None] * scales 
jvec= [None] * scales

for k in range(scales):
    [eig[k], nL[k]] = eigen_image_p(P.lpimages[k] , Gi,ratio**(k))
    # extract regional max and block out borders (edge effect)

    # generate mask for border
    border_mask[k] = np.zeros_like(eig[k],dtype=bool)
    border_mask[k][border:-border,border:-border]=True

    #E(k).regmask = imregionalmax(E(k).eig)
    regmask[k] = maximum_filter(eig[k],size=local_maxima_nhood)<=eig[k]

    regmask[k] = np.logical_and(regmask[k],border_mask[k])

    #[ivec[k], jvec[k]] = np.nonzero(regmask[k]) #coordinates of 1s in regmask
    # Just to match matlab version, can be reverted to optimise
    [jvec[k], ivec[k]] = np.nonzero(regmask[k].T)
    
# INITIALIZE feature positions and scales at highest level
# at highest resolution coordinates of features:
Fivec = ivec[0]
Fjvec = jvec[0]
Fsvec = np.ones_like(Fivec) #initial scale 
Fevec = eig[0][ivec[0],jvec[0]] #access the elements of eig at locations given by ivec,jvec

#i,j position of feature at the characteristic scale
Fsivec = Fivec 
Fsjvec = Fjvec

nLvec = nL[0][ivec[0],jvec[0]]
pivec = Fivec
pjvec = Fjvec
pind = np.array(list(range(len(Fivec))))
k = 1

mx = (np.floor(cols*ratio)-1)/(cols-1)  #scale conversion to next level
my = (np.floor(rows*ratio)-1)/(rows-1) 

[rows,cols]  = eig[k].shape #dimensions of next level
pendreg = np.zeros_like(pivec)
#sivec = np.round((pivec-1)*my+1).astype(int) #next scale ivec
#sjvec = np.round((pjvec-1)*mx+1).astype(int) #next scale jvec
# match matlab output
sivec = np.round(pivec*my).astype(int) #next scale ivec
sjvec = np.round(pjvec*mx).astype(int) #next scale jvec

csivec = sivec
csjvec = sjvec
cloc = sivec+1 + rows*(sjvec)
'''
for u in range(-1,2):  #account for motion of feature points between scales
        for v = in range(-1,2):

        soivec = sivec+v; %next scale ivec
        sojvec = sjvec+u; %next scale jvec

        loc = soivec + rows*(sojvec-1); #index vector of next level
        #have to account for motion of feature point at different
        #scales - more than just the sampling

        uvpend = E(k).regmask(loc) == 1;
        pendreg = pendreg | uvpend;

        cloc(uvpend) = loc(uvpend);
        csivec(uvpend) = soivec(uvpend);
        csjvec(uvpend) = sojvec(uvpend);

pend = pendreg & (E(k).nL(cloc) >= nLvec);

pind = pind(pend);

F.svec(pind) = k; %scale is k or larger
F.evec(pind) = E(k).eig(cloc(pend)); %eigen value is given at
                                     %level k or larger
F.sivec(pind) = csivec(pend);
F.sjvec(pind) = csjvec(pend);

pivec = csivec(pend);
pjvec = csjvec(pend);
nLvec = E(k).nL(cloc(pend));
k = k+1;
'''     
  


In [ ]:
print (sivec[-10:])
print (sjvec[-10:])
print (cloc[0:10])

In [ ]:
rows*(sjvec)

In [ ]:
pind

In [ ]:
Fevec[-10:]

In [ ]:
jvec[0][:10]

In [ ]:
Fevec = eig[0](ivec[0] + rows*(jvec[0]-1)) #1D index

In [ ]:
a = np.random.random((6,6))
a

In [ ]:
i = np.array([2,3,4,5]);
j = np.array([1,2,3,4]);

In [ ]:
a[i,j]